In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
base_dir = os.getcwd()  # Current working directory

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def plot(y, columns, file_path, save_path):
    # Read the Excel file into a DataFrame
    df_block = pd.read_excel(file_path)
    
    # Clean up the DataFrame and select required columns
    df_block_cleaned = df_block[["Coordinate y", "Sigma PSD", "Transform 2D", columns[0], columns[1], "SNR Mean (dB)", "Total Variation", "BRISQUE", "Clip-IQA"]]
    
    # Filter the DataFrame based on the provided 'y' value
    df = df_block_cleaned[df_block_cleaned["Coordinate y"] == y] 

    # Define the scores to plot
    scores = ["SNR Mean (dB)", "Total Variation", "BRISQUE", "Clip-IQA"]

    # Create a 2x2 grid for subplots
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Group data by setup
    groups = df.groupby([columns[0], columns[1], "Transform 2D"])

    # Define color maps for the two transformations
    colors_dct = plt.cm.Reds(np.linspace(0.7, 1, len(groups)))  # Red color map for dct
    colors_bior = plt.cm.Blues(np.linspace(0.7, 1, len(groups)))  # Blue color map for bior1.5

    # Iterate over scores and subplots
    for i, score in enumerate(scores):
        ax = axes[i // 2, i % 2]  # Determine subplot position
        blue_handles = []  # List to store blue legend handles
        red_handles = []   # List to store red legend handles
        blue_labels = []   # List to store blue legend labels
        red_labels = []    # List to store red legend labels

        for (wiener, ht, Trans2D), group in groups:
            if Trans2D == "dct":
                color = colors_dct[groups.get_group((wiener, ht, Trans2D)).index[0] % len(colors_dct)]
                label = f"T2D={Trans2D}, Wiener={wiener}, HT={ht}"
                line, = ax.plot(group["Sigma PSD"], group[score], marker="o", label=label, color=color)
                red_handles.append(line)  # Add handle to red_handles
                red_labels.append(label)   # Add label to red_labels
            elif Trans2D == "bior1.5":
                color = colors_bior[groups.get_group((wiener, ht, Trans2D)).index[0] % len(colors_bior)]
                label = f"T2D={Trans2D}, Wiener={wiener}, HT={ht}"
                line, = ax.plot(group["Sigma PSD"], group[score], marker="o", label=label, color=color)
                blue_handles.append(line)  # Add handle to blue_handles
                blue_labels.append(label)  # Add label to blue_labels
        
        # Set titles and labels
        ax.set_title(score)
        ax.set_xlabel("Sigma PSD")
        ax.set_ylabel("Score")
        ax.grid(True)

        # Combine the blue and red legend entries, blue first, then red
        handles = blue_handles + red_handles
        labels = blue_labels + red_labels
        if i == 0:
            ax.legend(handles, labels, loc='center left', bbox_to_anchor=(-1.2, 0.5))

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Save the plot to a file
    plt.savefig(save_path)

    # Close the figure to prevent it from being displayed
    plt.close(fig)


# Plot the results

In [4]:
file_path = f"{base_dir}/Results/Leech results/Threshold_Hard_Wiener/results_data_Threshold_Hard_Wiener.xlsx"
for y in range(15,23):
    save_path = f"{base_dir}/Results/Leech results/Threshold_Hard_Wiener/{y}_result_plot.png"
    plot(y, ["Tau Match", "Tau Match Wiener"],file_path, save_path)

In [5]:
file_path = f"{base_dir}/Results/resTarget results/Patch_Size/results_data_Patch_Size.xlsx"
for y in range(15,23):
    save_path = f"{base_dir}/Results/resTarget results/Patch_Size/{y}_result_plot.png"
    plot(y, ["Block Size HT", "Block Size Wiener"], file_path, save_path)
